In [ ]:
// Package references and using statements
#r "nuget:Microsoft.Azure.EventHubs, 4.1.0"
#r "nuget:Newtonsoft.Json, 12.0.3"
#r "nuget:Bogus, 28.4.4"

using System;
using System.Threading;
using Microsoft.Azure.EventHubs;
using System.Dynamic;
using System.Text;
using Newtonsoft.Json;
using Bogus;
using System.Collections.Generic;
using System.Linq;

In [ ]:
// Set up bogus classes

public class Address
{
    [JsonProperty("id")]
    public string Id { get; set; }
    [JsonProperty("customerId")]
    public string CustomerId { get; set; }
    [JsonProperty("streetAddress1")]
    public string StreetAddress1 { get; set; }
    [JsonProperty("streetAddress2")]
    public string StreetAddress2 { get; set; }
    [JsonProperty("city")]
    public string City { get; set; }
    [JsonProperty("state")]
    public string State { get; set; }
    [JsonProperty("zipcode")]
    public string Zipcode { get; set; }
}

public class Customer
{
    [JsonProperty("id")]
    public string Id { get; set; }
    [JsonProperty("firstName")]
    public string FirstName { get; set; }
    [JsonProperty("lastName")]
    public string LastName { get; set; }
    [JsonProperty("email")]
    public string Email { get; set; }
    [JsonProperty("phoneNumber")]
    public string PhoneNumber { get; set; }
    [JsonProperty("addresses")]
    public IEnumerable<Address> Addresses { get; set; }
}


In [ ]:
// Bogus test Data instantiation

Randomizer.Seed = new Random(8675309);

var testAddress = new Faker<Address>()
    .RuleFor(address => address.Id, (fake, address) => fake.Random.Guid().ToString())
    .RuleFor(address => address.CustomerId, (fake, address) => fake.Random.Guid().ToString())
    .RuleFor(address => address.StreetAddress1, (fake, address) => fake.Address.StreetAddress(true))
    .RuleFor(address => address.StreetAddress2, (fake, address) => fake.Address.StreetAddress(true))
    .RuleFor(address => address.City, (fake, address) => fake.Address.City())
    .RuleFor(address => address.State, (fake, address) => fake.Address.State())
    .RuleFor(address => address.Zipcode, (fake, address) => fake.Address.ZipCode(address.Zipcode));


var testCustomer = new Faker<Customer>()
    .RuleFor(cust => cust.Id, (fake, cust) => fake.Random.Guid().ToString())
    .RuleFor(cust => cust.FirstName, (fake, cust) => fake.Name.FirstName())
    .RuleFor(cust => cust.LastName, (fake, cust) => fake.Name.LastName())
    .RuleFor(cust => cust.Email, (fake, cust) => fake.Internet.ExampleEmail(cust.Email))
    .RuleFor(cust => cust.PhoneNumber, (fake, cust) => fake.Phone.PhoneNumber(cust.PhoneNumber))
    .RuleFor(cust => cust.Addresses, (fake, cust) => testAddress.Generate(3).Select(addr => 
    {
        // Replace fake customer ID with ID of the current Bogus customer
        addr.CustomerId = cust.Id;
        return addr;
    }));

In [ ]:
// Set up EventHubs client

EventHubClient eventHubClient;
var eventHubConnectionString = "Endpoint=sb://testsuffix-ns.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=2ZEATu8J4i5KZk1amnvaNemflRg8TNhJJYqk+qXpCdQ=";
var eventHubName = "test";

var connectionStringBuilder = new EventHubsConnectionStringBuilder(eventHubConnectionString) 
{
    EntityPath = eventHubName
};

eventHubClient = EventHubClient.CreateFromConnectionString(connectionStringBuilder.ToString());

In [ ]:
public async Task SendMessageToEventHub(string jsonMessage)
{
    var encodedMessage = Encoding.UTF8.GetBytes(jsonMessage);
    var eventData = new EventData(encodedMessage);
    await eventHubClient.SendAsync(eventData);
}

public void FlushDataList(IList dataList)
{
    dataList.Clear();
    display("Data list flushed");
}

In [ ]:
List<Customer> dataList = new List<Customer>();

var customerEntity = testCustomer.Generate();
dataList.Add(customerEntity);

foreach(var entity in dataList)
{
    var jsonMessage = JsonConvert.SerializeObject(customerEntity);
    await SendMessageToEventHub(jsonMessage);
}

display(dataList);

In [ ]:
FlushDataList(dataList);